#Objective: To fillup the nulls in Locality variable

1. I have used GEOPY package to extract suburb/city/hamlet names from the address extracted from the latitude and longitude to fill the nulls in 'Locality'

(But there were still 4 nulls as geopy could not extract suburb info for certain data)

2. The remaining nulls were filled up using the suburb names extracted directly from the 'Address' in cleaned_incidents dataset. 

In [ ]:
import pandas as pd
!pip install geopandas
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re
import nltk 

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')

In [3]:
#cols = ['Address','Lat', 'Long', 'Locality', 'Category']
df = pd.read_csv('/content/drive/MyDrive/ESV Data/cleaned_incidents1.csv')
df = df.dropna(subset=['Category'])
df

,ActionTaken,Address,AssetLabel,CauseCommunity,CauseEnvironment,CausePre,CauseTechnical,CauseWorkP,ContactType,CorrectProtection,EventDescription,FailedAssets,FailedExplosion,FailedOilFilled,FailedOtherAssets,FailedOtherAssetsOther,FeederNumber,IncidentCause,IncidentConsequence,IncidentDatetime,IncidentFireFFactorReportable,IncidentFireSeverity,IncidentID,IncidentLocationType,IncidentLocationTypeOther,IncidentNumber,IncidentType,Lat,Long,MadeSafe,NetworkType,Status,SubmissionID,SubmittedDateTimeString,Voltage,WeatherStation,Postcode,Locality,Category
0,Crew isolated supply and undertook repairs,"PARA PARK, 1490 HENDY MAIN ROAD, PARAPARAP VIC...",NaN,NaN,NaN,NaN,Earth fault,NaN,NaN,NaN,A nearby customer reported sparking of electri...,Conductor (ABC),0,0,0,NaN,NaN,HVABC cable faulted midspan,Grassfire,5/10/2015 20:36,1.0,Small: 10 - 1000 sq.m,56,Roadway,NaN,20151009PWA_1,Infrastructure (network-based),-38.288982,144.191502,1,Powercor,Report,1030,01:04.8,22kV AC,Avalon Airport,3240.0,Paraparap,OH Cable
1,Crew undertook repairs,"1154-1198 CHRISTIES ROAD, RAVENHALL VIC 3023",NaN,Working too close to underground cables,NaN,NaN,NaN,NaN,NaN,NaN,A contractor reported that he had contacted an...,No failed asset,0,0,0,NaN,NaN,Contractor contacted earthing conductor,No Go Zone (contact),7/10/2015 11:00,NaN,NaN,64,NaN,NaN,20151012PWA_5,Infrastructure (network-based),-37.767377,144.738509,1,Powercor,Report,96,39:29.0,Earthing cable,Laverton Raaf,3023.0,Ravenhall,Dug up
2,Crew isolated supply and undertook repairs,"326 DUNBARS ROAD, PETERBOROUGH VIC 3270",NaN,NaN,NaN,NaN,Corrosion,NaN,NaN,1.0,A field crew attending an outage found that a ...,Conductor (bare),0,0,0,NaN,NaN,Conductor broke due to rust,Serious risk to public safety (e.g. live condu...,10/10/2015 8:36,NaN,NaN,67,Agricultural,NaN,20151013PWA_1,Infrastructure (network-based),-38.586647,142.901137,1,Powercor,Report,86,45:20.1,22kV AC,Warrnambool Airport Ndb,3270.0,Peterborough,Connection
3,Crew undertook repairs,"41-49 BANK STREET, SOUTH MELBOURNE VIC 3205",NaN,Vandalism,NaN,NaN,NaN,NaN,NaN,1.0,Interfere and vandalism in substation. Unknown...,Indoor,0,2,0,NaN,NaN,Vandalism,Loss of supply;Damage to network assets,4/10/2015 10:30,NaN,NaN,73,NaN,NaN,20151015PWA_4,Infrastructure (network-based),-37.832352,144.968452,1,CitiPower,Report,92,03:46.9,22kV AC,Essendon Airport,3205.0,South Melbourne,Other
4,Crew isolated supply and undertook repairs,"199 WILSONS ROAD, WHITTINGTON VIC 3219",NaN,Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,A nearby customer reported that a high load ha...,Service conductor,0,0,0,NaN,NaN,Unknown high load contacted LV service cable,No Go Zone (contact);Damage to network assets,5/10/2015 14:41,NaN,NaN,77,NaN,NaN,20151016PWA_1,Infrastructure (network-based),-38.177436,144.390480,1,Powercor,Report,98,13:07.0,Low voltage AC (<1kV),Avalon Airport,3219.0,Whittington,Vehicle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6499,The crew replaced a 3 phase FOLCB on the pole,"301 ST KILDA STREET, BRIGHTON VIC 3186",1806162,NaN,NaN,NaN,Earth fault,NaN,NaN,2.0,A report came in to UE of a FMB sparking at 31...,Connection box,0,0,0,NaN,NB33,The likely cause was determined by EIO to be a...,Plant/equipment,9/06/2020 18:39,1.0,Negligible: no ground fire,15718,Residential,NaN,20200611UTD_02,Infrastructure (network-based),-37.905307,144.988061,1,United Energy Distribution,Report,19935,19:54.4,Low voltage AC (<1kV),Moorabbin Airport,3186.0,Brighton,Connection
6500,Conductor repaired at point of contact.,"48 Banks Rd, Mount Wallace VIC 3342, Australia",30555208,Working too close to overhead lines,NaN,NaN,NaN,NaN,NaN,NaN,Report received from a resident to advise that...,Conductor (bare),2,2,2,NaN,BMH003,Crane struck line,No Go Zone (contact);Damage to network assets,12/06/2020 14:02,NaN,NaN,15744,Agricultural,NaN,20200615PWA_01,Infrastructure (network-based),-37.707726,144.199888,1,P

In [4]:
df.isnull().sum()
#Locality has 15 nulls

ActionTaken                         0
Address                             0
AssetLabel                       1334
CauseCommunity                   4023
CauseEnvironment                 4983
CausePre                         6411
CauseTechnical                   3338
CauseWorkP                       6327
ContactType                      5637
CorrectProtection                2251
EventDescription                    0
FailedAssets                        0
FailedExplosion                     0
FailedOilFilled                     0
FailedOtherAssets                   0
FailedOtherAssetsOther           6479
FeederNumber                     1326
IncidentCause                       1
IncidentConsequence                 0
IncidentDatetime                    0
IncidentFireFFactorReportable    3831
IncidentFireSeverity             3728
IncidentID                          0
IncidentLocationType              650
IncidentLocationTypeOther        6427
IncidentNumber                      0
IncidentType

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6489 entries, 0 to 6503
Data columns (total 39 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ActionTaken                    6489 non-null   object 
 1   Address                        6489 non-null   object 
 2   AssetLabel                     5155 non-null   object 
 3   CauseCommunity                 2466 non-null   object 
 4   CauseEnvironment               1506 non-null   object 
 5   CausePre                       78 non-null     object 
 6   CauseTechnical                 3151 non-null   object 
 7   CauseWorkP                     162 non-null    object 
 8   ContactType                    852 non-null    object 
 9   CorrectProtection              4238 non-null   float64
 10  EventDescription               6489 non-null   object 
 11  FailedAssets                   6489 non-null   object 
 12  FailedExplosion                6489 non-null   i

In [6]:
#concatenate the lat and longitude to one column 
df['geom'] = df['Lat'].map(str) + ',' + df['Long'].map(str)

In [7]:
df['geom']

0                 -38.288982,144.191502
1        -37.76737670000001,144.7385093
2               -38.5866468,142.9011365
3                 -37.832352,144.968452
4                  -38.177436,144.39048
                     ...               
6499    -37.90530677,144.98806100000004
6500              -37.707726,144.199888
6501              -34.978769,142.365388
6502              -35.478065,143.228477
6503      -38.117825,144.91773999999995
Name: geom, Length: 6489, dtype: object

In [8]:
#storing the data where the Locality is null as a seperate dataframe
df1 = df[df['Locality'].isnull()]
df1

,ActionTaken,Address,AssetLabel,CauseCommunity,CauseEnvironment,CausePre,CauseTechnical,CauseWorkP,ContactType,CorrectProtection,EventDescription,FailedAssets,FailedExplosion,FailedOilFilled,FailedOtherAssets,FailedOtherAssetsOther,FeederNumber,IncidentCause,IncidentConsequence,IncidentDatetime,IncidentFireFFactorReportable,IncidentFireSeverity,IncidentID,IncidentLocationType,IncidentLocationTypeOther,IncidentNumber,IncidentType,Lat,Long,MadeSafe,NetworkType,Status,SubmissionID,SubmittedDateTimeString,Voltage,WeatherStation,Postcode,Locality,Category,geom
104,Local Field Team Manager has arranged for Ther...,"Firmins Ln, Hazelwood North, VIC 3840",2615189,Bird/ caused flashover.;Other,Bird (other);Bird (unknown),NaN,Insulator,No work practice factor,Phase to phase,1.0,CEOT - contacted fault crew and advised of pol...,Melted insulation off HV conductor bridge from...,0,0,0,NaN,mwts32,"Conductor bridge covered by poly insulation, w...",Grassfire,8/11/2015 12:00,NaN,Low: Less than 5 sqm,205,Roadway,NaN,20151112SPN_2,Infrastructure (network-based),-38.247950,146.431531,1,AusNet Services Distribution,Report,317,25:07.0,22kV AC,Morwell (Latrobe Valley Airport),3840.0,NaN,Pole,"-38.24795,146.431531"
107,Crew isolated supply and undertook repairs,"166 MOORE ROAD, FAIRY DELL (NORTH) VIC 3561",NaN,NaN,NaN,NaN,Leakage,NaN,NaN,1.0,A field crew attending an outage found that a ...,Crossarm (wood),0,0,0,NaN,NaN,Crossarm fire,Pole-top fire,31/10/2015 8:26,NaN,Negligible: no ground fire,219,Roadway,NaN,20151113PWA_13,Infrastructure (network-based),-36.336559,144.732009,1,Powercor,Report,332,21:58.8,22kV AC,Bendigo Airport,3561.0,NaN,Crossarm,"-36.336559,144.732009"
927,Repair works completed -LV fuse box has been r...,"40 MADDISON LANE,GREENDALE,VIC,3341",105696,NaN,NaN,NaN,Overheated connection,NaN,NaN,2.0,A customer reported a fire on a pole at the in...,Any other type;Connection box,0,0,0,NaN,BMH003,Overheated connection,Loss of supply;Grassfire,15/06/2016 20:11,1.0,Localised: less than 10 sq.m,2481,Roadway,NaN,20160617PWA_01,Infrastructure (network-based),-37.558597,144.306180,1,Powercor,Report,3213,54:28.2,Low voltage AC (<1kV),Avalon Airport,3341.0,NaN,Connection,"-37.558597,144.30618"
1354,Replaced Krone box and restored supply,"Lot 1 Signboard Lane, Dunston",NaN,NaN,NaN,NaN,Unknown technical factor,NaN,NaN,2.0,IMS 223842. Ground fire reported at base of po...,Connection box,0,2,0,NaN,sle31,Spoke to fault crew and there was no apparent ...,Grassfire,22/08/2016 2:40,1.0,Localised: less than 10 sq.m,2793,NaN,NaN,20160823SPN_01,Infrastructure (network-based),-38.221164,147.274490,1,AusNet Services Distribution,Report,3832,21:23.1,Low voltage AC (<1kV),Bairnsdale Airport,NaN,NaN,Connection,"-38.22116424,147.27449019999997"
1719,On arrival crew commenced repairs.,"MALLEE HIGHWAY, LINGA Vic 3509",32139791,NaN,Wind;Fallen,NaN,NaN,NaN,NaN,1.0,VicFire reported wire down and ground fire at ...,Conductor (bare),2,2,2,NaN,OYN001,Tree from outside clearance space blown over i...,Loss of supply;Grassfire,28/12/2016 8:35,1.0,Medium: 1000 sq.m - 10 ha,4349,Roadway,NaN,20161229PWA_04,Infrastructure (network-based),-35.166426,141.741460,1,Powercor,Report,12417,39:56.0,22kV AC,Mildura Airport,3509.0,NaN,Trees,"-35.166426,141.74146000000005"
4421,Site visit confirmed fire was extinguished pri...,"MURRAY VALLEY HIGHWAY, HUON 3695",5214084,NaN,Bird (other),NaN,NaN,NaN,Phase to earth,2.0,Fire at base of pole,Conductor (bare),0,0,0,NaN,NaN,Found 2 dead magpies at base of pole and cente...,Grassfire,13/01/2019 21:20,1.0,Small: 10 - 1000 sq.m,10935,Roadway,NaN,20190114SPN_01,Infrastructure (network-based),-36.258839,147.092258,1,AusNet Services Distribution,Report,17766,03:10.1,22kV AC,Wangaratta Aero,3695.0,NaN,Animal,"-36.25883901,147.09225800000004"
4449,"Crew attended, isolated and repaired cable","79 CITY VISTA COURT,PLUMPTON,VIC,3335",30117188,Working too close to underground cables,NaN,NaN,NaN,NaN,NaN,2.0,Customer advised that an excavator has dug up ...

In [9]:
from nltk.corpus import stopwords
#remove the regex and stopwords from address column
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub(' ', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('vic', ' ')
    text = text.replace('rd', ' ')
    text = text.replace('st', ' ')


    return text

df1['Address'] = df1['Address'].apply(clean_text)
df1['Address'] = df1['Address'].str.replace('\d+', '')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
#splitup the address to check if there is occurence of suburb names under one single column
df1['Address'].str.split(' ', expand = True)

,0,1,2,3,4,5,6,7,8,9,10,11
104,firmins,ln,,hazelwood,north,,,,,None,None,None
107,,moore,road,,fairy,dell,,north,,,,
927,,maddison,lane,greendale,,,,None,None,None,None,None
1354,lot,,signboa,,lane,,dun,on,None,None,None,None
1719,mallee,highway,,linga,,,,None,None,None,None,None
4421,murray,valley,highway,,huon,,None,None,None,None,None,None
4449,,city,vi,a,court,plumpton,,,,None,None,None
4541,,glamis,,,,greensborough,None,None,None,None,None,None
4585,,frieda,ct,ferntree,gully,,,,None,None,None,None
4758,,,,,princes,highway,corio,,None,None,None,None


In [11]:
points = df1.geom
points.index

Int64Index([ 104,  107,  927, 1354, 1719, 4421, 4449, 4541, 4585, 4758, 4761,
            4993, 5006, 5051, 5059, 5060, 5180, 5184, 5194, 5200, 5206, 5310,
            5487, 5527, 5627, 5725, 5727, 5729, 5865, 5867, 6032, 6075, 6149,
            6181, 6253, 6300, 6318, 6364, 6372, 6391, 6403, 6413, 6429, 6431,
            6461],
           dtype='int64')

In [12]:
points = df1.geom
suburbs = []

locator = Nominatim(user_agent='myGeocoder')

#use geopy geocoder to extract the address out of the geom 
for point in points[0:45]:
    location = locator.reverse(point)
    raw = location.raw
    sub = raw['address']
    #df1['Locality'][points.index] = sub
    #df1['Locality'] = df1['Locality'].fillna(sub)
    suburbs.append(sub)

In [13]:
dfn = pd.DataFrame(suburbs)
dfn = dfn.drop(columns=['road','state', 'postcode','country','country_code','village', 'house_number','amenity', 'industrial', 'shop','residential'])

#fill the nulls with suburb/city/hamlet index from the extracted address
dfn['suburb'] = dfn['suburb'].fillna(dfn['hamlet'])
dfn['suburb'] = dfn['suburb'].fillna(dfn['city'])
dfn['suburb'] = dfn['suburb'].fillna(dfn['municipality'])

dfn['suburb'].isnull().sum()
#we still get 4 nulls in column - suburb

4

In [14]:
df1['subs'] = dfn['suburb'].tolist()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
df1['Address'].str.split(' ', expand = True)
#.drop(columns = [0,1,2,3,4,5,7,8,9,10,11])
#fill the nulls with extracted suburb names from split address
df1['subs'] = df1['subs'].fillna(df1['Address'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
print(df1['subs'].isnull().sum())
df['Locality'] = df['Locality'].fillna(df1['subs'])
df['Locality'].isnull().sum()
#df = df.drop(columns = ['geom'])
df.to_csv('cleaned_incidentsnew.csv')

0
